# Performance dataframes

In [1]:
import os
import re
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

/Users/andersvandvik/Repositories/masters-thesis/venv/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## General parameters

In [2]:
instance_key = 'instance'

avg_constr_obj_key = 'constr obj'
best_obj_key = 'best obj'
worst_obj_key = 'worst obj'
avg_obj_key = 'obj'

max_time_key = 'max time'
min_time_key = 'min time'
avg_time_key = 'time'

max_iter_key = 'max iter'
min_iter_key = 'min iter'
avg_best_sol_iter_key = 'best found iter'
avg_iter_key = 'iter'

set_part_key = 'sp improv (#)'
ls_improv_key = 'ls improv (#)'
best_ls_improv_percent_key = 'best ls improv (%)'
dr_improv_key = 'dr improv (#)'

best_sol_found_by_key = 'best sol found by'

gap_key = 'gap'
preprocess_key = 'preproc'
model_key = 'model time'
variables_key = 'variables'

project_path = os.path.dirname(os.path.abspath('.'))
directory_path_alns = '/output/solstorm/alns/performance/'
directory_path_exact = '/output/solstorm/arcflow/performance/'

generate_df = True
run_number = 'fifth'

## Functions

In [45]:
def map_instance_to_data_alns(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
            
            avg_constr_obj = history_json['construction_heuristic_objective']
            obj = history_json['best_objective']
            time = history_json['runtime']
            it = history_json['number_of_iterations']
            best_it = history_json['best_sol_found_in_iteration']
            set_part_improv = history_json['number_of_improvements_by_set_partitioning']
            ls_improv = history_json['number_of_improvements_by_local_search']
            ls_improv_percent = history_json['best_improvement_local_search']
            dr_improv = history_json['number_of_improvements_by_destroy_repair']
            best_sol_found_by = history_json['best_solution_found_by']

            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]

                if obj < data[0]:
                    data[0] = obj
                if obj > data[1]:
                    data[1] = obj
                if time > data[4]:
                    data[4] = time
                if time < data[5]:
                    data[5] = time
                if it > data[7]:
                    data[7] = it
                if it < data[8]:
                    data[8] = it
                
                data[2] += avg_constr_obj
                data[3] += obj
                data[6] += time
                data[9] += best_it
                data[10] += it
                data[11] += set_part_improv
                data[12] += ls_improv
                data[13] += ls_improv_percent
                data[14] += dr_improv
                data[15].append(best_sol_found_by)
                data[16] += 1
            else:
                instance_to_data[instance_name] = [obj, obj, avg_constr_obj, obj, 
                                                   time, time, time, 
                                                   it, it, best_it, it, 
                                                   set_part_improv, ls_improv, ls_improv_percent, dr_improv,
                                                   [best_sol_found_by],
                                                   1]
    return instance_to_data

def generate_run_df_alns(run_name):
    run_path = project_path + directory_path_alns + run_name
    instance_to_data = map_instance_to_data_alns(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               best_obj_key, worst_obj_key, avg_constr_obj_key, avg_obj_key, 
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_best_sol_iter_key, avg_iter_key,
                               set_part_key, ls_improv_key, best_ls_improv_percent_key, dr_improv_key,
                               best_sol_found_by_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        
        nbr_sims = data[16]
        if nbr_sims != 5:
            print(f'{instance} DEVIATES IN SIMULATIONS!')
        
        best_objective = data[0]
        worst_objective = data[1]
        avg_constr_objective = data[2] / nbr_sims
        avg_objective = data[3] / nbr_sims
        max_time = data[4]
        min_time = data[5]
        avg_time = data[6] / nbr_sims
        max_iter = data[7]
        min_iter = data[8]
        avg_best_sol_iter = data[9] / nbr_sims
        avg_iter = data[10] / nbr_sims
        avg_set_part_improv = data[11] / nbr_sims
        avg_ls_improv = data[12] / nbr_sims
        avg_ls_improv_percent = data[13] / nbr_sims
        avg_dr_improv = data[14] / nbr_sims
        best_sol_found_by = data[15]
        
        row = pd.Series({instance_key: instance, 
                         best_obj_key: best_objective,
                         worst_obj_key: worst_objective,
                         avg_constr_obj_key: avg_constr_objective,
                         avg_obj_key: avg_objective,
                         max_time_key: max_time,
                         min_time_key: min_time,
                         avg_time_key: avg_time, 
                         max_iter_key: max_iter,
                         min_iter_key: min_iter,
                         avg_best_sol_iter_key: avg_best_sol_iter,
                         avg_iter_key: avg_iter,
                         set_part_key: avg_set_part_improv,
                         ls_improv_key: avg_ls_improv,
                         best_ls_improv_percent_key: avg_ls_improv_percent,
                         dr_improv_key: avg_dr_improv,
                         best_sol_found_by_key: best_sol_found_by})
        
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_constr_obj_key: df[avg_constr_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_best_sol_iter_key: df[avg_best_sol_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean(),
                          ls_improv_key: df[ls_improv_key].mean(),
                          best_ls_improv_percent_key: df[best_ls_improv_percent_key].mean(),
                          dr_improv_key: df[dr_improv_key].mean(),
                          best_sol_found_by_key: 'Manual'})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def aggregate_df_by_instance_group_alns(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        best_obj = row[best_obj_key]
        worst_obj = row[worst_obj_key]
        avg_constr_obj = row[avg_constr_obj_key]
        avg_obj = row[avg_obj_key]
        max_time = row[max_time_key]
        min_time = row[min_time_key]
        avg_time = row[avg_time_key]
        max_iter = row[max_iter_key]
        min_iter = row[min_iter_key]
        avg_best_sol_iter = row[avg_best_sol_iter_key]
        avg_iter = row[avg_iter_key]
        avg_set_part_improv = row[set_part_key]
        avg_ls_improv = row[ls_improv_key]
        avg_ls_improv_percent = row[best_ls_improv_percent_key]
        avg_dr_improv = row[dr_improv_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += best_obj
            data[1] += worst_obj
            data[2] += avg_constr_obj
            data[3] += avg_obj
            data[4] += max_time
            data[5] += min_time
            data[6] += avg_time
            data[7] += max_iter
            data[8] += min_iter
            data[9] += avg_best_sol_iter
            data[10] += avg_iter
            data[11] += avg_set_part_improv
            data[12] += avg_ls_improv
            data[13] += avg_ls_improv_percent
            data[14] += avg_dr_improv
            data[15] += 1  # Number of times encountered instance size
        else:
            data = [best_obj, worst_obj, avg_constr_obj, avg_obj, 
                    max_time, min_time, avg_time, 
                    max_iter, min_iter, avg_best_sol_iter, avg_iter, 
                    avg_set_part_improv, avg_ls_improv, avg_ls_improv_percent, avg_dr_improv, 1]
            instance_size_to_data[instance_size] = data

    df = pd.DataFrame(columns=['instance_group', 
                               best_obj_key, worst_obj_key, avg_constr_obj_key, avg_obj_key, 
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_best_sol_iter_key, avg_iter_key,
                               set_part_key, ls_improv_key, best_ls_improv_percent_key, dr_improv_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[15]
        if nbr_sims != 5:
            print(f'{instance_size} INSTANCE_SIZE DEVIATES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         best_obj_key: data[0] / nbr_sims,
                         worst_obj_key: data[1] / nbr_sims,
                         avg_constr_obj_key: data[2] / nbr_sims,
                         avg_obj_key: data[3] / nbr_sims, 
                         max_time_key: data[4] / nbr_sims,
                         min_time_key: data[5] / nbr_sims,
                         avg_time_key: data[6] / nbr_sims, 
                         max_iter_key: data[7] / nbr_sims,
                         min_iter_key: data[8] / nbr_sims,
                         avg_best_sol_iter_key: data[9] / nbr_sims,
                         avg_iter_key: data[10] / nbr_sims,
                         set_part_key: data[11] / nbr_sims,
                         ls_improv_key: data[12] / nbr_sims,
                         best_ls_improv_percent_key: data[13] / nbr_sims,
                         dr_improv_key: data[14] / nbr_sims})
    
        df = df.append(row, ignore_index=True)
        
    mean_row = pd.Series({'instance_group': 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_constr_obj_key: df[avg_constr_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_best_sol_iter_key: df[avg_best_sol_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean(),
                          ls_improv_key: df[ls_improv_key].mean(),
                          best_ls_improv_percent_key: df[best_ls_improv_percent_key].mean(),
                          dr_improv_key: df[dr_improv_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def map_instance_to_data_exact(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        with open(run_path + file_name) as file:
            exact_json = json.load(file)

        obj = exact_json['objective']['objective_bound']
        gap = exact_json['objective']['optimality_gap']
        model_runtime = exact_json['runtime']['model_runtime']
        preprocess_runtime = exact_json['runtime']['preprocess_runtime']
        variables = exact_json['variables']['number_of_variables']
        
        if instance_name in instance_to_data:
            print('Multiple versions of same instance!')
        
        instance_to_data[instance_name] = [obj, gap, model_runtime, preprocess_runtime, variables]
    
    return instance_to_data

def generate_run_df_exact(run_name):
    run_path = project_path + directory_path_exact + run_name
    instance_to_data = map_instance_to_data_exact(run_path)

    df = pd.DataFrame(columns=[instance_key, avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    for instance in instance_to_data:
        data = instance_to_data[instance]
        obj = data[0]
        gap = data[1]
        preprocess_runtime = data[2]
        model_runtime = data[3]
        variables = data[4]
        row = pd.Series({instance_key: instance,
                         avg_obj_key: obj,
                         gap_key: gap,
                         preprocess_key: preprocess_runtime,
                         model_key: model_runtime,
                         variables_key: variables})
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(),
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, avg_constr_obj_key,
                 max_time_key, min_time_key, 
                 max_iter_key, min_iter_key], 
                axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    if drop:
        li = [i for i in range(5, len(df_total.columns), 5)]
        df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

def aggregate_df_by_instance_group_exact(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        obj = row[avg_obj_key]
        gap = row[gap_key]
        preprocess_runtime = row[preprocess_key]
        model_runtime = row[model_key]
        variables = row[variables_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += obj
            data[1] += gap
            data[2] += preprocess_runtime
            data[3] += model_runtime
            data[4] += variables
            data[5] += 1 # Number of times encountered instance size
        else:
            instance_size_to_data[instance_size] = [obj, gap, preprocess_runtime, model_runtime, variables, 1]

    df = pd.DataFrame(columns=['instance_group', avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[5]
        if nbr_sims < 5:
            print(f'{instance_size} INSTANCE_SIZE LESS THAN FIVE INSTANCES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         avg_obj_key: data[0] / nbr_sims,
                         gap_key: data[1] / nbr_sims,
                         preprocess_key: data[2] / nbr_sims, 
                         model_key: data[3] / nbr_sims,
                         variables_key: data[4] / nbr_sims})
    
        df = df.append(row, ignore_index=True)

        
    mean_row = pd.Series({'instance_group': 'Mean values', 
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(), 
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def load_df(file_name):
    run_df = pd.read_pickle(f'dataframes/performance/{file_name}')
    # run_df = sort_df(run_df, sort_column)
    return run_df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, avg_constr_obj_key,
                 max_time_key, min_time_key, 
                 max_iter_key, min_iter_key, avg_best_sol_iter_key], 
                axis=1, inplace=True)
        if best_sol_found_by_key in df:
            df.drop([best_sol_found_by_key], axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    
    
    # Drop duplicate instance columns
    if drop:
        li = [i for i in range(8, len(df_total.columns), 8)]
        df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

## ALNS baseline

In [46]:
if generate_df:
    run_baseline_name = f'{run_number}/baseline/'
    run_baseline_df = generate_run_df_alns(run_baseline_name)
    run_baseline_agg_df = aggregate_df_by_instance_group_alns(run_baseline_df)

    run_baseline_file_name = f'dataframes/performance/baseline.pkl'
    run_baseline_agg_file_name = f'dataframes/performance/baseline_agg.pkl'
    run_baseline_df.to_pickle(run_baseline_file_name)
    run_baseline_agg_df.to_pickle(run_baseline_agg_file_name)

5-7-1-1 DEVIATES IN SIMULATIONS!


In [6]:
run_baseline_df = load_df('baseline.pkl')
run_baseline_df

,instance,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#),best sol found by
0,5-5-1-1,2519.9,2519.9,2852.9,2519.9,0.7,0.7,0.7,5000,5000,4.2,5000.0,0.0,0.0,0.0,1.6,"[destroy_repair, destroy_repair, destroy_repai..."
1,5-5-1-2,2214.2,2214.2,2222.8,2214.2,0.7,0.7,0.7,5000,5000,0.2,5000.0,0.0,0.0,0.0,0.2,"[construction_heuristic, construction_heuristi..."
2,5-6-1-1,1939.6,1939.6,1939.6,1939.6,0.9,0.8,0.9,5000,5000,0.0,5000.0,0.0,0.0,0.0,0.0,"[construction_heuristic, construction_heuristi..."
3,5-7-1-1,2709.7,2709.7,2896.6,2709.7,1.3,1.3,1.3,5000,5000,21.8,5000.0,0.0,0.0,0.0,1.2,"[destroy_repair, destroy_repair, destroy_repai..."
4,5-7-1-2,1704.6,1704.6,1713.2,1704.6,1.2,1.1,1.1,5000,5000,1.4,5000.0,0.0,0.0,0.0,0.4,"[construction_heuristic, construction_heuristi..."
5,7-8-1-1,2046.9,2046.9,2161.3,2046.9,2.0,1.7,1.8,5000,5000,1.8,5000.0,0.0,0.0,0.0,1.4,"[destroy_repair, destroy_repair, construction_..."
6,7-8-1-2,1971.3,1971.3,2059.1,1971.3,2.0,1.7,1.8,5000,5000,12.8,5000.0,0.0,0.0,0.0,1.6,"[destroy_repair, destroy_repair, destroy_repai..."
7,7-8-1-3,2080.2,2080.2,2222.7,2080.2,2.0,1.9,2.0,5000,5000,3.4,5000.0,0.0,0.0,0.0,3.4,"[destroy_repair, destroy_repair, destroy_repai..."
8,7-9-1-1,2170.4,2170.4,2170.4,2170.4,4.5,4.2,4.4,5000,5000,0.2,5000.0,0.0,0.0,0.0,0.2,"[construction_heuristic, construction_heuristi..."
9,7-9-1-2,2206.1,2206.1,2206.1,2206.1,2.8,2.1,2.5,5000,5000,0.0,5000.0,0.0,0.0,0.0,0.0,"[construction_heuristic, construction_heuristi..."


In [7]:
run_baseline_agg_df = load_df('baseline_agg.pkl')
run_baseline_agg_df

,instance_group,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#)
0,5,2217.6,2217.6,2325.0,2217.6,1.0,0.9,0.9,5000.0,5000.0,5.5,5000.0,0.0,0.0,0.0,0.7
1,7,2095.0,2095.0,2163.9,2095.0,2.7,2.3,2.5,5000.0,5000.0,3.6,5000.0,0.0,0.0,0.0,1.3
2,9,5627.5,5627.5,13438.6,5627.5,4.8,4.1,4.5,5000.0,5000.0,51.5,5000.0,0.0,0.0,0.0,4.9
3,11,3517.5,3517.5,4424.7,3517.5,19.8,17.5,18.9,5000.0,5000.0,147.8,5000.0,0.0,0.0,0.0,6.4
4,13,3970.5,3984.8,6163.9,3973.4,34.8,30.2,32.3,5000.0,5000.0,261.5,5000.0,0.0,0.0,0.0,8.8
5,15,8375.4,8429.6,13143.2,8404.3,39.2,34.6,37.2,5000.0,5000.0,1012.4,5000.0,0.0,0.0,0.0,12.5
6,17,4967.1,4992.0,6487.4,4974.7,61.7,52.5,56.7,5000.0,5000.0,1096.0,5000.0,0.0,0.0,0.0,10.7
7,19,5153.5,5177.4,8906.4,5164.1,76.5,66.2,70.9,5000.0,5000.0,1107.8,5000.0,0.0,0.0,0.0,13.7
8,21,9301.3,9358.7,15335.5,9322.4,93.2,77.6,84.8,5000.0,5000.0,1828.7,5000.0,0.0,0.0,0.0,16.7
9,23,5967.6,6032.9,8773.0,6001.0,139.2,123.1,131.7,5000.0,5000.0,1421.9,5000.0,0.0,0.0,0.0,13.7


## Sequential ALNS

In [49]:
if generate_df and False:
    run_sequential_name = 'first/sequential/'
    run_sequential_df = generate_run_df_alns(run_sequential_name)
    run_sequential_agg_df = aggregate_df_by_instance_group_alns(run_sequential_df)

    run_sequential_file_name = f'dataframes/performance/sequential.pkl'
    run_sequential_agg_file_name = f'dataframes/performance/sequential_agg.pkl'
    run_sequential_df.to_pickle(run_sequential_file_name)
    run_sequential_agg_df.to_pickle(run_sequential_agg_file_name)

In [50]:
run_sequential_df = load_df('sequential.pkl')
run_sequential_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.7,0.7,0.7,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.8,0.8,0.8,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.4,1.3,1.3,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.2,1.2,1.2,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.2,1.9,2.1,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,5.6,5.4,5.5,5000,5000,5000.0,0.0
7,7-8-1-3,2071.9,2071.9,2071.9,3.6,3.4,3.5,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.8,4.7,4.8,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,6.7,6.3,6.6,5000,5000,5000.0,0.0


In [51]:
run_sequential_agg_df = load_df('sequential_agg.pkl')
run_sequential_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.0,0.9,0.9,5000.0,5000.0,5000.0,0.0
1,7,2093.3,2093.3,2093.3,4.6,4.3,4.5,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,10.8,9.5,10.1,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3517.5,29.9,26.0,28.0,5000.0,5000.0,5000.0,0.0
4,13,3970.5,3970.5,3970.5,67.4,56.4,61.7,5000.0,5000.0,5000.0,0.0
5,15,6179.0,8399.9,7944.4,86.7,74.9,80.7,5000.0,5000.0,5000.0,0.0
6,17,4827.9,4986.8,4943.2,151.6,124.0,138.8,5000.0,5000.0,5000.0,0.0
7,19,4926.9,5173.1,5095.8,190.6,166.2,178.9,5000.0,5000.0,5000.0,0.0
8,21,8362.7,9332.5,9116.9,238.3,201.3,218.0,5000.0,5000.0,5000.0,0.0
9,23,5619.4,5995.6,5871.9,361.2,326.2,342.6,5000.0,5000.0,5000.0,0.0


## LNS

In [52]:
if generate_df:
    run_lns_name = f'{run_number}/lns/'
    run_lns_df = generate_run_df_alns(run_lns_name)
    run_lns_agg_df = aggregate_df_by_instance_group_alns(run_lns_df)

    run_lns_file_name = f'dataframes/performance/lns.pkl'
    run_lns_agg_file_name = f'dataframes/performance/lns_agg.pkl'
    run_lns_df.to_pickle(run_lns_file_name)
    run_lns_agg_df.to_pickle(run_lns_agg_file_name)

9-9-1-2 DEVIATES IN SIMULATIONS!


In [53]:
run_lns_df = load_df('lns.pkl')
run_lns_df

,instance,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#),best sol found by
0,5-5-1-1,2519.9,2519.9,2852.9,2519.9,0.8,0.7,0.7,5000,5000,7.4,5000.0,0.0,0.0,0.0,2.0,"[destroy_repair, destroy_repair, destroy_repai..."
1,5-5-1-2,2214.2,2214.2,2240.1,2214.2,0.7,0.7,0.7,5000,5000,2.2,5000.0,0.0,0.0,0.0,0.6,"[construction_heuristic, destroy_repair, destr..."
2,5-6-1-1,1939.6,1939.6,1939.6,1939.6,0.9,0.8,0.9,5000,5000,0.0,5000.0,0.0,0.0,0.0,0.0,"[construction_heuristic, construction_heuristi..."
3,5-7-1-1,2709.7,2709.7,2896.6,2709.7,1.3,1.2,1.3,5000,5000,14.8,5000.0,0.0,0.0,0.0,1.2,"[destroy_repair, destroy_repair, destroy_repai..."
4,5-7-1-2,1704.6,1704.6,1713.2,1704.6,1.1,1.1,1.1,5000,5000,1.8,5000.0,0.0,0.0,0.0,0.4,"[construction_heuristic, construction_heuristi..."
5,7-8-1-1,2046.9,2046.9,2123.1,2046.9,1.9,1.8,1.9,5000,5000,2.0,5000.0,0.0,0.0,0.0,1.2,"[construction_heuristic, construction_heuristi..."
6,7-8-1-2,1971.3,1971.3,2047.5,1971.3,1.8,1.6,1.7,5000,5000,8.0,5000.0,0.0,0.0,0.0,1.8,"[destroy_repair, destroy_repair, destroy_repai..."
7,7-8-1-3,2080.2,2080.2,2259.9,2080.2,2.0,1.8,2.0,5000,5000,9.4,5000.0,0.0,0.0,0.0,3.0,"[destroy_repair, destroy_repair, destroy_repai..."
8,7-9-1-1,2170.4,2170.4,2170.4,2170.4,4.5,4.1,4.3,5000,5000,0.0,5000.0,0.0,0.0,0.0,0.0,"[construction_heuristic, construction_heuristi..."
9,7-9-1-2,2206.1,2206.1,2206.1,2206.1,2.8,2.2,2.5,5000,5000,0.0,5000.0,0.0,0.0,0.0,0.0,"[construction_heuristic, construction_heuristi..."


In [54]:
run_lns_agg_df = load_df('lns_agg.pkl')
run_lns_agg_df

,instance_group,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#)
0,5,2217.6,2217.6,2328.5,2217.6,1.0,0.9,0.9,5000.0,5000.0,5.2,5000.0,0.0,0.0,0.0,0.8
1,7,2095.0,2095.0,2161.4,2095.0,2.6,2.3,2.5,5000.0,5000.0,3.9,5000.0,0.0,0.0,0.0,1.2
2,9,5627.5,5627.5,13434.3,5627.5,5.0,4.1,4.5,5000.0,5000.0,64.0,5000.0,0.0,0.0,0.0,4.9
3,11,3517.5,3517.5,4352.1,3517.5,20.0,17.6,18.6,5000.0,5000.0,151.8,5000.0,0.0,0.0,0.0,7.3
4,13,3970.5,3974.6,5964.1,3971.3,34.8,30.1,32.1,5000.0,5000.0,423.0,5000.0,0.0,0.0,0.0,9.5
5,15,8377.1,8435.1,13368.8,8405.2,39.3,34.0,36.5,5000.0,5000.0,1147.2,5000.0,0.0,0.0,0.0,14.3
6,17,4968.5,5002.4,7118.5,4978.5,61.6,52.7,57.3,5000.0,5000.0,976.7,5000.0,0.0,0.0,0.0,11.1
7,19,5153.5,5185.8,8630.8,5168.7,79.3,62.6,71.4,5000.0,5000.0,1158.2,5000.0,0.0,0.0,0.0,13.0
8,21,9262.2,9388.0,15371.1,9321.2,89.5,75.0,82.6,5000.0,5000.0,1624.8,5000.0,0.0,0.0,0.0,15.4
9,23,5959.6,6043.4,8854.7,5992.7,138.3,117.6,129.3,5000.0,5000.0,1555.8,5000.0,0.0,0.0,0.0,14.6


## ALNS + local search

In [55]:
if generate_df:
    run_ls_name = f'{run_number}/ls/'
    run_ls_df = generate_run_df_alns(run_ls_name)
    run_ls_agg_df = aggregate_df_by_instance_group_alns(run_ls_df)

    run_ls_file_name = f'dataframes/performance/ls.pkl'
    run_ls_agg_file_name = f'dataframes/performance/ls_agg.pkl'
    run_ls_df.to_pickle(run_ls_file_name)
    run_ls_agg_df.to_pickle(run_ls_agg_file_name)

23-31-4-1 DEVIATES IN SIMULATIONS!


In [56]:
run_ls_df = load_df('ls.pkl')
run_ls_df

,instance,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#),best sol found by
0,11-12-2-1,3587.5,3587.5,4632.1,3587.5,35.6,31.7,33.6,5000,5000,32.6,5000.0,0.0,3.6,0.1,1.6,"[destroy_repair, destroy_repair, local_search,..."
1,11-12-2-2,3788.9,3788.9,4640.6,3788.9,44.9,38.9,43.3,5000,5000,152.2,5000.0,0.0,4.2,0.1,2.4,"[destroy_repair, destroy_repair, destroy_repai..."
2,11-13-2-1,3470.8,3470.8,3734.0,3470.8,52.3,46.0,49.4,5000,5000,19.8,5000.0,0.0,2.0,0.1,1.4,"[local_search, local_search, local_search, des..."
3,11-14-2-1,3710.8,3710.8,4336.6,3710.8,52.9,42.8,48.9,5000,5000,133.6,5000.0,0.0,4.6,0.1,1.6,"[destroy_repair, destroy_repair, destroy_repai..."
4,11-15-2-1,3029.7,3029.7,4107.4,3029.7,28.3,23.1,26.3,5000,5000,45.6,5000.0,0.0,3.4,0.1,2.6,"[local_search, destroy_repair, destroy_repair,..."
5,13-14-2-1,3681.9,3681.9,4710.9,3681.9,68.9,56.7,64.7,5000,5000,16.4,5000.0,0.0,3.6,0.1,0.0,"[local_search, local_search, local_search, loc..."
6,13-15-2-1,4122.6,4122.6,8500.5,4122.6,96.6,78.5,87.2,5000,5000,320.0,5000.0,0.0,6.4,0.2,1.4,"[local_search, destroy_repair, local_search, l..."
7,13-15-2-2,3779.4,3779.4,5548.0,3779.4,47.7,41.5,44.6,5000,5000,27.2,5000.0,0.0,4.2,0.2,0.8,"[local_search, local_search, destroy_repair, l..."
8,13-16-2-1,4239.6,4239.6,5869.4,4239.6,82.8,75.3,78.4,5000,5000,226.0,5000.0,0.0,5.2,0.2,1.8,"[destroy_repair, destroy_repair, local_search,..."
9,13-18-2-1,4029.1,4029.1,5234.8,4029.1,110.0,96.9,102.5,5000,5000,136.4,5000.0,0.0,4.4,0.2,2.6,"[destroy_repair, destroy_repair, destroy_repai..."


In [57]:
run_ls_agg_df = load_df('ls_agg.pkl')
run_ls_agg_df

,instance_group,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#)
0,11,3517.5,3517.5,4290.1,3517.5,42.8,36.5,40.3,5000.0,5000.0,76.8,5000.0,0.0,3.6,0.1,1.9
1,13,3970.5,3970.5,5972.7,3970.5,81.2,69.8,75.5,5000.0,5000.0,145.2,5000.0,0.0,4.8,0.2,1.3
2,15,8375.4,8387.9,13254.1,8379.9,96.3,79.8,89.1,5000.0,5000.0,920.6,5000.0,0.0,6.8,0.2,1.7
3,17,4945.2,4968.6,7170.7,4962.9,194.9,174.8,184.6,5000.0,5000.0,824.2,5000.0,0.0,8.7,0.2,1.0
4,19,5153.5,5153.5,8644.7,5153.5,261.3,225.3,243.5,5000.0,5000.0,1271.5,5000.0,0.0,9.3,0.2,1.3
5,21,9273.5,9329.4,15447.3,9303.4,230.8,197.4,213.6,5000.0,5000.0,1935.0,5000.0,0.0,9.9,0.2,1.6
6,23,5963.7,5979.5,8804.3,5973.1,474.5,436.8,456.4,4715.8,4495.0,2254.2,4621.6,0.0,12.2,0.1,0.3
7,Mean values,5885.6,5901.0,9083.4,5894.4,197.4,174.3,186.1,4959.4,4927.9,1061.0,4945.9,0.0,7.9,0.2,1.3


## ALNS + set partitioning

In [58]:
if generate_df: 
    run_sp_name = f'{run_number}/sp/'
    run_sp_df = generate_run_df_alns(run_sp_name)
    run_sp_agg_df = aggregate_df_by_instance_group_alns(run_sp_df)

    run_sp_file_name = f'dataframes/performance/sp.pkl'
    run_sp_agg_file_name = f'dataframes/performance/sp_agg.pkl'
    run_sp_df.to_pickle(run_sp_file_name)
    run_sp_agg_df.to_pickle(run_sp_agg_file_name)

19-21-3-2 DEVIATES IN SIMULATIONS!


In [59]:
run_sp_df = load_df('sp.pkl')
run_sp_df

,instance,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#),best sol found by
0,5-5-1-1,2519.9,2519.9,2834.3,2519.9,0.8,0.8,0.8,5000,5000,14.2,5000.0,0.0,0.0,0.0,2.0,"[destroy_repair, destroy_repair, destroy_repai..."
1,5-5-1-2,2214.2,2214.2,2389.1,2214.2,0.8,0.7,0.7,5000,5000,1.0,5000.0,0.0,0.0,0.0,1.0,"[destroy_repair, destroy_repair, destroy_repai..."
2,5-6-1-1,1939.6,1939.6,1939.6,1939.6,0.9,0.9,0.9,5000,5000,0.0,5000.0,0.0,0.0,0.0,0.0,"[construction_heuristic, construction_heuristi..."
3,5-7-1-1,2709.7,2709.7,2896.6,2709.7,1.5,1.3,1.4,5000,5000,14.6,5000.0,0.0,0.0,0.0,1.2,"[destroy_repair, destroy_repair, destroy_repai..."
4,5-7-1-2,1704.6,1704.6,1717.5,1704.6,1.4,1.3,1.3,5000,5000,2.2,5000.0,0.0,0.0,0.0,1.0,"[construction_heuristic, construction_heuristi..."
5,7-8-1-1,2046.9,2046.9,2123.1,2046.9,2.2,1.9,2.1,5000,5000,1.6,5000.0,0.0,0.0,0.0,1.2,"[construction_heuristic, construction_heuristi..."
6,7-8-1-2,1971.3,1971.3,2046.1,1971.3,2.2,1.8,2.0,5000,5000,9.4,5000.0,0.0,0.0,0.0,1.4,"[destroy_repair, destroy_repair, destroy_repai..."
7,7-8-1-3,2080.2,2080.2,2259.9,2080.2,2.2,2.1,2.2,5000,5000,3.6,5000.0,0.0,0.0,0.0,2.8,"[destroy_repair, destroy_repair, destroy_repai..."
8,7-9-1-1,2170.4,2170.4,2170.4,2170.4,4.9,4.7,4.8,5000,5000,0.0,5000.0,0.0,0.0,0.0,0.0,"[construction_heuristic, construction_heuristi..."
9,7-9-1-2,2206.1,2206.1,2206.1,2206.1,3.2,2.4,2.7,5000,5000,0.0,5000.0,0.0,0.0,0.0,0.0,"[construction_heuristic, construction_heuristi..."


In [18]:
run_sp_agg_df = load_df('sp_agg.pkl')
run_sp_agg_df

,instance_group,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#)
0,5,2217.6,2217.6,2355.4,2217.6,1.1,1.0,1.0,5000.0,5000.0,6.4,5000.0,0.0,0.0,0.0,1.0
1,7,2095.0,2095.0,2161.1,2095.0,2.9,2.6,2.8,5000.0,5000.0,2.9,5000.0,0.0,0.0,0.0,1.1
2,9,5627.5,5627.5,13445.3,5627.5,5.9,5.1,5.5,5000.0,5000.0,67.6,5000.0,0.2,0.0,0.0,5.4
3,11,3517.5,3517.5,4426.1,3517.5,25.1,21.4,23.2,5000.0,5000.0,108.6,5000.0,0.0,0.0,0.0,7.2
4,13,3970.5,3984.8,6028.9,3973.4,50.3,42.9,47.1,5000.0,5000.0,238.0,5000.0,0.1,0.0,0.0,8.7
5,15,8375.4,8392.2,13182.8,8386.7,63.9,49.5,56.7,5000.0,5000.0,598.3,5000.0,1.1,0.1,0.0,10.2
6,17,4966.1,4987.5,6963.3,4972.1,105.0,82.6,95.5,5000.0,5000.0,629.6,5000.0,1.4,0.0,0.0,11.9
7,19,5153.5,5171.6,8620.5,5159.2,146.0,120.9,132.0,5000.0,5000.0,754.6,5000.0,1.6,0.0,0.0,13.6
8,21,9255.8,9318.9,15486.5,9286.0,172.3,147.4,161.2,5000.0,5000.0,1650.5,5000.0,2.4,0.0,0.0,14.9
9,23,5959.4,5998.9,8639.9,5971.9,248.1,208.0,225.7,5000.0,5000.0,1668.3,5000.0,2.2,0.0,0.0,14.6


## ALNS + local search + set partitioning

In [19]:
if generate_df:
    run_lssp_name = f'{run_number}/lssp/'
    run_lssp_df = generate_run_df_alns(run_lssp_name)
    run_lssp_agg_df = aggregate_df_by_instance_group_alns(run_lssp_df)

    run_lssp_file_name = f'dataframes/performance/lssp.pkl'
    run_lssp_agg_file_name = f'dataframes/performance/lssp_agg.pkl'
    run_lssp_df.to_pickle(run_lssp_file_name)
    run_lssp_agg_df.to_pickle(run_lssp_agg_file_name)

21 INSTANCE_SIZE DEVIATES!


In [20]:
run_lssp_df = load_df('lssp.pkl')
run_lssp_df

,instance,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#),best sol found by
0,11-12-2-1,3587.5,3587.5,4632.1,3587.5,45.4,33.8,39.6,5000,5000,26.0,5000.0,0.0,4.4,0.1,2.8,"[destroy_repair, destroy_repair, destroy_repai..."
1,11-12-2-2,3788.9,3788.9,4571.6,3788.9,57.7,50.8,54.8,5000,5000,160.2,5000.0,0.0,4.4,0.1,2.2,"[destroy_repair, destroy_repair, destroy_repai..."
2,11-13-2-1,3470.8,3470.8,3753.3,3470.8,72.5,59.2,65.2,5000,5000,23.8,5000.0,0.0,3.2,0.1,0.8,"[local_search, destroy_repair, local_search, l..."
3,11-14-2-1,3710.8,3710.8,4323.7,3710.8,64.8,55.6,61.1,5000,5000,136.4,5000.0,0.0,4.4,0.1,2.2,"[destroy_repair, destroy_repair, destroy_repai..."
4,11-15-2-1,3029.7,3029.7,4463.2,3029.7,42.6,26.9,34.2,5000,5000,47.4,5000.0,0.0,4.6,0.1,1.8,"[destroy_repair, local_search, destroy_repair,..."
5,13-14-2-1,3681.9,3681.9,4716.5,3681.9,101.6,84.4,92.4,5000,5000,32.2,5000.0,0.0,4.0,0.1,0.6,"[local_search, destroy_repair, local_search, l..."
6,13-15-2-1,4122.6,4122.6,8401.2,4122.6,145.7,125.4,138.5,5000,5000,188.0,5000.0,0.0,5.6,0.2,0.6,"[local_search, destroy_repair, local_search, l..."
7,13-15-2-2,3779.4,3779.4,7170.1,3779.4,63.6,47.7,56.4,5000,5000,25.4,5000.0,0.0,3.6,0.1,1.6,"[local_search, local_search, local_search, des..."
8,13-16-2-1,4239.6,4239.6,5843.8,4239.6,160.0,124.2,141.2,5000,5000,232.6,5000.0,0.0,4.4,0.2,2.2,"[destroy_repair, local_search, destroy_repair,..."
9,13-18-2-1,4029.1,4029.1,5208.3,4029.1,243.2,203.2,221.0,5000,5000,116.6,5000.0,0.2,4.2,0.2,3.4,"[destroy_repair, destroy_repair, local_search,..."


In [21]:
run_lssp_agg_df = load_df('lssp_agg.pkl')
run_lssp_agg_df

,instance_group,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#)
0,11,3517.5,3517.5,4348.8,3517.5,56.6,45.3,51.0,5000.0,5000.0,78.8,5000.0,0.0,4.2,0.1,2.0
1,13,3970.5,3970.5,6268.0,3970.5,142.8,117.0,129.9,5000.0,5000.0,119.0,5000.0,0.0,4.4,0.2,1.7
2,15,8375.4,8375.4,13131.2,8375.4,176.6,142.2,158.2,5000.0,5000.0,234.0,5000.0,0.5,6.6,0.2,2.0
3,17,4966.1,4968.5,7488.0,4966.6,428.6,336.4,381.3,5000.0,4800.0,341.6,4960.0,1.0,6.0,0.2,2.2
4,19,5151.9,5157.8,8650.6,5154.1,503.5,472.1,486.1,4147.8,3727.8,735.4,3927.9,1.0,8.4,0.2,2.2
5,21,5591.6,5591.6,9389.5,5591.6,356.9,332.8,345.2,5000.0,5000.0,1227.4,5000.0,0.8,8.0,0.1,1.2
6,Mean values,5262.2,5263.6,8212.7,5262.6,277.5,241.0,258.6,4858.0,4754.6,456.0,4814.6,0.6,6.3,0.2,1.9


## Exact solver

In [116]:
if generate_df:
    run_exact_name = 'results/'
    run_exact_df = generate_run_df_exact(run_exact_name)
    run_exact_agg_df = aggregate_df_by_instance_group_exact(run_exact_df)

    run_exact_file_name = f'dataframes/performance/exact.pkl'
    run_exact_agg_file_name = f'dataframes/performance/exact_agg.pkl'
    run_exact_df.to_pickle(run_exact_file_name)
    run_exact_agg_df.to_pickle(run_exact_agg_file_name)

21 INSTANCE_SIZE LESS THAN FIVE INSTANCES!


In [117]:
run_exact_df = load_df('exact.pkl')
run_exact_df

,instance,obj,gap,preproc,model time,variables
0,5-5-1-1,2519.9,0.0,1.3,1.5,24808
1,5-5-1-2,2214.2,0.0,3.7,1.5,36406
2,5-6-1-1,1939.6,0.0,4.2,1.8,40610
3,5-7-1-1,2709.7,0.0,2.7,2.3,35034
4,5-7-1-2,1704.6,0.0,16.0,2.7,64346
5,7-8-1-1,2046.9,0.0,15.2,3.5,71164
6,7-8-1-2,1971.3,0.0,65.4,3.7,84072
7,7-8-1-3,2071.9,0.0,9.1,3.0,47880
8,7-9-1-1,2170.4,0.0,26.6,3.5,80958
9,7-9-1-2,2206.1,0.0,38.7,3.7,85362


In [118]:
run_exact_agg_df = load_df('exact_agg.pkl')
run_exact_agg_df

,instance_group,obj,gap,preproc,model time,variables
0,5,2217.6,0.0,5.6,2.0,40240.8
1,7,2093.3,0.0,31.0,3.5,73887.2
2,9,5627.5,0.0,43.7,5.7,124820.4
3,11,3450.2,0.0,2015.2,13.7,309915.6
4,13,3717.5,0.1,3493.6,20.3,447488.4
5,15,7257.5,0.1,3231.8,27.4,606759.0
6,17,3677.1,0.4,3600.8,45.0,984075.2
7,19,3877.3,0.8,3601.8,52.3,1135832.8
8,21,5603.5,0.6,3600.3,61.7,1344890.0
9,Mean values,4169.1,0.2,2180.4,25.7,563101.0


## Parallel vs. sequential heuristics

In [119]:
baseline_sequential_df = merge_dfs([run_baseline_df, run_sequential_df], True)
baseline_sequential_agg_df = merge_dfs([run_baseline_agg_df, run_sequential_agg_df], True)

In [120]:
baseline_sequential_df

,instance,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5-5-1-1,2519.9,0.7,5000.0,0.0,2519.9,0.7,5000.0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.0,2214.2,0.7,5000.0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,1939.6,0.8,5000.0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,0.0,2709.7,1.3,5000.0,0.0
4,5-7-1-2,1704.6,1.2,5000.0,0.0,1704.6,1.2,5000.0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,0.0,2046.9,2.1,5000.0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,0.0,1971.3,5.5,5000.0,0.0
7,7-8-1-3,2080.2,2.0,5000.0,0.0,2071.9,3.5,5000.0,0.0
8,7-9-1-1,2170.4,4.3,5000.0,0.0,2170.4,4.8,5000.0,0.0
9,7-9-1-2,2206.1,2.3,5000.0,0.0,2206.1,6.6,5000.0,0.0


In [121]:
baseline_sequential_agg_df

,instance_group,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5,2217.6,1.0,5000.0,0.0,2217.6,0.9,5000.0,0.0
1,7,2095.0,2.4,5000.0,0.0,2093.3,4.5,5000.0,0.0
2,9,5627.5,4.5,5000.0,0.0,5627.5,10.1,5000.0,0.0
3,11,3517.5,18.8,5000.0,0.0,3517.5,28.0,5000.0,0.0
4,13,3970.5,32.5,5000.0,0.0,3970.5,61.7,5000.0,0.0
5,15,8389.0,37.3,5000.0,0.0,7944.4,80.7,5000.0,0.0
6,17,4976.7,57.9,5000.0,0.0,4943.2,138.8,5000.0,0.0
7,19,5167.4,72.6,5000.0,0.0,5095.8,178.9,5000.0,0.0
8,21,9118.2,80.3,4800.0,0.0,9116.9,218.0,5000.0,0.0
9,23,5983.2,130.5,5000.0,0.0,5871.9,342.6,5000.0,0.0


## ALNS vs. LNS

In [60]:
baseline_lns_df = merge_dfs([run_baseline_df, run_lns_df], True)
baseline_lns_agg_df = merge_dfs([run_baseline_agg_df, run_lns_agg_df], True)

In [61]:
baseline_lns_df

,instance,obj,time,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#),obj,time,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#)
0,5-5-1-1,2519.9,0.7,5000.0,0.0,0.0,0.0,1.6,2519.9,0.7,5000.0,0.0,0.0,0.0,2.0
1,5-5-1-2,2214.2,0.7,5000.0,0.0,0.0,0.0,0.2,2214.2,0.7,5000.0,0.0,0.0,0.0,0.6
2,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,0.0,0.0,0.0,1.2,2709.7,1.3,5000.0,0.0,0.0,0.0,1.2
4,5-7-1-2,1704.6,1.1,5000.0,0.0,0.0,0.0,0.4,1704.6,1.1,5000.0,0.0,0.0,0.0,0.4
5,7-8-1-1,2046.9,1.8,5000.0,0.0,0.0,0.0,1.4,2046.9,1.9,5000.0,0.0,0.0,0.0,1.2
6,7-8-1-2,1971.3,1.8,5000.0,0.0,0.0,0.0,1.6,1971.3,1.7,5000.0,0.0,0.0,0.0,1.8
7,7-8-1-3,2080.2,2.0,5000.0,0.0,0.0,0.0,3.4,2080.2,2.0,5000.0,0.0,0.0,0.0,3.0
8,7-9-1-1,2170.4,4.4,5000.0,0.0,0.0,0.0,0.2,2170.4,4.3,5000.0,0.0,0.0,0.0,0.0
9,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0


In [62]:
baseline_lns_agg_df

,instance_group,obj,time,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#),obj,time,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#)
0,5,2217.6,0.9,5000.0,0.0,0.0,0.0,0.7,2217.6,0.9,5000.0,0.0,0.0,0.0,0.8
1,7,2095.0,2.5,5000.0,0.0,0.0,0.0,1.3,2095.0,2.5,5000.0,0.0,0.0,0.0,1.2
2,9,5627.5,4.5,5000.0,0.0,0.0,0.0,4.9,5627.5,4.5,5000.0,0.0,0.0,0.0,4.9
3,11,3517.5,18.9,5000.0,0.0,0.0,0.0,6.4,3517.5,18.6,5000.0,0.0,0.0,0.0,7.3
4,13,3973.4,32.3,5000.0,0.0,0.0,0.0,8.8,3971.3,32.1,5000.0,0.0,0.0,0.0,9.5
5,15,8404.3,37.2,5000.0,0.0,0.0,0.0,12.5,8405.2,36.5,5000.0,0.0,0.0,0.0,14.3
6,17,4974.7,56.7,5000.0,0.0,0.0,0.0,10.7,4978.5,57.3,5000.0,0.0,0.0,0.0,11.1
7,19,5164.1,70.9,5000.0,0.0,0.0,0.0,13.7,5168.7,71.4,5000.0,0.0,0.0,0.0,13.0
8,21,9322.4,84.8,5000.0,0.0,0.0,0.0,16.7,9321.2,82.6,5000.0,0.0,0.0,0.0,15.4
9,23,6001.0,131.7,5000.0,0.0,0.0,0.0,13.7,5992.7,129.3,5000.0,0.0,0.0,0.0,14.6


## ALNS vs. ALNS + local search

In [125]:
baseline_ls_df = merge_dfs([run_baseline_df, run_ls_df], True)
baseline_ls_agg_df = merge_dfs([run_baseline_agg_df, run_ls_agg_df], True)

In [126]:
baseline_ls_df

,instance,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5-5-1-1,2519.9,0.7,5000.0,0.0,2519.9,1.5,5000.0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.0,2214.2,1.3,5000.0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,1939.6,1.6,5000.0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,0.0,2709.7,2.2,5000.0,0.0
4,5-7-1-2,1704.6,1.2,5000.0,0.0,1704.6,1.9,5000.0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,0.0,2046.9,4.4,5000.0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,0.0,1971.3,4.0,5000.0,0.0
7,7-8-1-3,2080.2,2.0,5000.0,0.0,2080.2,4.6,5000.0,0.0
8,7-9-1-1,2170.4,4.3,5000.0,0.0,2170.4,7.3,5000.0,0.0
9,7-9-1-2,2206.1,2.3,5000.0,0.0,2206.1,5.2,5000.0,0.0


In [127]:
baseline_ls_agg_df

,instance_group,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5,2217.6,1.0,5000.0,0.0,2217.6,1.7,5000.0,0.0
1,7,2095.0,2.4,5000.0,0.0,2095.0,5.1,5000.0,0.0
2,9,5627.5,4.5,5000.0,0.0,5627.5,8.6,5000.0,0.0
3,11,3517.5,18.8,5000.0,0.0,3517.5,52.5,5000.0,0.0
4,13,3970.5,32.5,5000.0,0.0,3970.5,91.5,5000.0,0.0
5,15,8389.0,37.3,5000.0,0.0,8383.1,92.5,5000.0,0.0
6,17,4976.7,57.9,5000.0,0.0,4967.6,186.6,5000.0,0.0
7,19,5167.4,72.6,5000.0,0.0,5156.6,252.7,5000.0,0.0
8,21,9118.2,80.3,4800.0,0.0,9315.2,211.8,5000.0,0.0
9,23,5983.2,130.5,5000.0,0.0,5971.7,440.7,4826.0,0.0


## ALNS vs. ALNS + set partitioning

In [63]:
baseline_sp_df = merge_dfs([run_baseline_df, run_sp_df], True)
baseline_sp_agg_df = merge_dfs([run_baseline_agg_df, run_sp_agg_df], True)

In [64]:
baseline_sp_df

,instance,obj,time,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#),obj,time,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#)
0,5-5-1-1,2519.9,0.7,5000.0,0.0,0.0,0.0,1.6,2519.9,0.8,5000.0,0.0,0.0,0.0,2.0
1,5-5-1-2,2214.2,0.7,5000.0,0.0,0.0,0.0,0.2,2214.2,0.7,5000.0,0.0,0.0,0.0,1.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,0.0,0.0,0.0,1.2,2709.7,1.4,5000.0,0.0,0.0,0.0,1.2
4,5-7-1-2,1704.6,1.1,5000.0,0.0,0.0,0.0,0.4,1704.6,1.3,5000.0,0.0,0.0,0.0,1.0
5,7-8-1-1,2046.9,1.8,5000.0,0.0,0.0,0.0,1.4,2046.9,2.1,5000.0,0.0,0.0,0.0,1.2
6,7-8-1-2,1971.3,1.8,5000.0,0.0,0.0,0.0,1.6,1971.3,2.0,5000.0,0.0,0.0,0.0,1.4
7,7-8-1-3,2080.2,2.0,5000.0,0.0,0.0,0.0,3.4,2080.2,2.2,5000.0,0.0,0.0,0.0,2.8
8,7-9-1-1,2170.4,4.4,5000.0,0.0,0.0,0.0,0.2,2170.4,4.8,5000.0,0.0,0.0,0.0,0.0
9,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,2206.1,2.7,5000.0,0.0,0.0,0.0,0.0


In [65]:
baseline_sp_agg_df

,instance_group,obj,time,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#),obj,time,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#)
0,5,2217.6,0.9,5000.0,0.0,0.0,0.0,0.7,2217.6,1.0,5000.0,0.0,0.0,0.0,1.0
1,7,2095.0,2.5,5000.0,0.0,0.0,0.0,1.3,2095.0,2.8,5000.0,0.0,0.0,0.0,1.1
2,9,5627.5,4.5,5000.0,0.0,0.0,0.0,4.9,5627.5,5.5,5000.0,0.2,0.0,0.0,5.4
3,11,3517.5,18.9,5000.0,0.0,0.0,0.0,6.4,3517.5,23.2,5000.0,0.0,0.0,0.0,7.2
4,13,3973.4,32.3,5000.0,0.0,0.0,0.0,8.8,3973.4,47.1,5000.0,0.1,0.0,0.0,8.7
5,15,8404.3,37.2,5000.0,0.0,0.0,0.0,12.5,8386.7,56.7,5000.0,1.1,0.1,0.0,10.2
6,17,4974.7,56.7,5000.0,0.0,0.0,0.0,10.7,4972.1,95.5,5000.0,1.4,0.0,0.0,11.9
7,19,5164.1,70.9,5000.0,0.0,0.0,0.0,13.7,5159.2,132.0,5000.0,1.6,0.0,0.0,13.6
8,21,9322.4,84.8,5000.0,0.0,0.0,0.0,16.7,9286.0,161.2,5000.0,2.4,0.0,0.0,14.9
9,23,6001.0,131.7,5000.0,0.0,0.0,0.0,13.7,5971.9,225.7,5000.0,2.2,0.0,0.0,14.6


## ALNS vs. ALNS + local search + set partitioning

In [131]:
baseline_lssp_df = merge_dfs([run_baseline_df, run_lssp_df], True)
baseline_lssp_agg_df = merge_dfs([run_baseline_agg_df, run_lssp_agg_df], True)

In [132]:
baseline_lssp_df

,instance,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5-5-1-1,2519.9,0.7,5000.0,0.0,3587.5,54.8,5000.0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.0,3788.9,62.6,5000.0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,3470.8,78.3,5000.0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,0.0,3710.8,78.9,5000.0,0.0
4,5-7-1-2,1704.6,1.2,5000.0,0.0,3029.7,47.8,5000.0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,0.0,3681.9,123.2,5000.0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,0.0,4122.6,121.0,5000.0,0.2
7,7-8-1-3,2080.2,2.0,5000.0,0.0,3779.4,71.9,5000.0,0.2
8,7-9-1-1,2170.4,4.3,5000.0,0.0,4239.6,134.4,5000.0,0.0
9,7-9-1-2,2206.1,2.3,5000.0,0.0,4029.1,252.2,5000.0,0.2


In [133]:
baseline_lssp_agg_df

,instance_group,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5,2217.6,1.0,5000.0,0.0,3517.5,64.5,5000.0,0.0
1,7,2095.0,2.4,5000.0,0.0,3970.5,140.5,5000.0,0.1
2,9,5627.5,4.5,5000.0,0.0,8375.4,146.8,5000.0,0.8
3,11,3517.5,18.8,5000.0,0.0,4966.6,441.6,4663.5,0.9
4,13,3970.5,32.5,5000.0,0.0,5154.4,503.5,2951.4,1.6
5,15,8389.0,37.3,5000.0,0.0,9240.5,479.6,4250.4,2.6
6,17,4976.7,57.9,5000.0,0.0,5962.3,600.7,2092.1,2.2
7,19,5167.4,72.6,5000.0,0.0,6063.6,553.0,1766.2,2.0
8,21,9118.2,80.3,4800.0,0.0,5906.4,366.3,3840.5,1.3
9,23,5983.2,130.5,5000.0,0.0,NaN,NaN,NaN,NaN
